If in VSCODE:

In [4]:

#install requirements if running locall by using:
# pip install -r requirements.txt
# then run:
# pip install git+https://github.com/huggingface/transformers
# to install latest version of transformers and get surpport for lLama

if in COLAB run this

In [5]:
#! pip install accelerate peft bitsandbytes  trl
#!pip install git+https://github.com/huggingface/transformers

Hyper params

running this cell is mandatory

In [2]:


# The model that you want to train from the Hugging Face hub
model_name = "codellama/CodeLlama-7b-Instruct-hf"

# The instruction dataset to use
dataset_name = "pechaut/cairo-instruct"

# Fine-tuned model name
new_model = "llama-2-7b-cairo-trained"

# Output directory where the model predicti
# ons and checkpoints will be stored
output_dir = "./results"

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "pechaut/codellama-cairo-instruct"

max_steps = 700 # to tweak to get the best out of the model 


Load Base Model and Tokenizer

In [2]:

import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token =tokenizer.eos_token

# load the quantized settings: 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)


# don't use the cache
model.config.use_cache = False
model.config.pretraining_tp=1
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )



Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Import Dataset from Hugging Face

In [3]:
from datasets import load_dataset

# Load the dataset
dataset_train = load_dataset(dataset_name, split="train", download_mode='force_redownload',ignore_verifications=True)
dataset_test = load_dataset(dataset_name, split="eval", download_mode='force_redownload',ignore_verifications=True)


/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/datasets/load.py:2097: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/197 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/197 [00:00<?, ? examples/s]

Check to see if everything is fine before launching training

In [4]:
df = dataset_train.to_pandas()
print(df["text"][18])

<s>[INST]In Cairo:  test if myArray is empty[/INST]
    if(myArry.is_empty) 'it is empty'.print()

 Explanations:

 - If you want to check if an array is empty or not, you can use the is_empty() method, which returns true if the array is empty and false otherwise.
 </s>


Load Model

In [5]:
from trl import SFTTrainer
import transformers



supervised_finetuning_trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=3,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        output_dir=new_model,
        max_steps=max_steps,
        fp16=True,
        push_to_hub=True,
    ),
)


/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/197 [00:00<?, ? examples/s]

/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [6]:
supervised_finetuning_trainer.train()


You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.299000
20,2.282000
30,1.966900
40,1.722200
50,1.596500
60,1.561100
70,1.359800
80,1.462600
90,1.430200
100,1.258800


TrainOutput(global_step=700, training_loss=0.7436608219146729, metrics={'train_runtime': 4262.8201, 'train_samples_per_second': 0.493, 'train_steps_per_second': 0.164, 'total_flos': 7931226612989952.0, 'train_loss': 0.7436608219146729, 'epoch': 10.66})

If you are on a colab, push to hub or save to drive

In [27]:
supervised_finetuning_trainer.push_to_hub()


NameError: name 'supervised_finetuning_trainer' is not defined

---

#Inference

---

- Load the adapter and create a model from it
- reuse the tokenizer

In [29]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=new_model, device_map="auto", 
    low_cpu_mem_usage=True,
    load_in_4bit=True,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(new_model)




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [51]:
#prompt = "Create an array and append some animal names"
#prompt="create an array 'messages' that contains a u128, a u32, a u256"
#prompt = "create a structure for mailAccount"
#prompt = "create an array of felt and append 1 to the array"
#prompt = "create a felt and affect it a value of 1"
#prompt="create a function for fibonacci"
#prompt = "what are spans used for"
prompt = "How do I know if an array is empty"
text =f"[INST]I'm working in Cairo. You are a cairo expert and know no other computer language, answer in less than 170 words{prompt} [/INST]"

input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids.cuda()

outputs = model.generate(input_ids=input_ids,       
    max_new_tokens=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.01,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
    )


print(f"Prompt:\n{prompt}\n")
print(f"Answer:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(text):]}")


Prompt:
How do I know if an array is empty

Answer:


To check if an array is empty, you can use the is_empty() method.

Here is an example:

    use debug::PrintTrait;

    fn main() {
        let arr = Array::<u64>::new();
        if arr.is_empty() {
            'the array is empty'.print();
        }
    }

Explanation:
- The line 'let arr = Array::<u64>::new();' creates an empty array
- The line 'if arr.is_empty()' checks if the array is empty
- If the array is empty, it prints 'the array is empty'
 
Note:
- If the array is empty, the if statement will be true, and the corresponding code will be executed.
- If the array is not empty, the if statement will be false, and the corresponding code will not be executed.


In [28]:
model.push_to_hub(hub_name)

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pechaut/codellama-cairo-instruct/commit/9c72d7dcc7609aa7bf7c0542aa6ba85eccea8a72', commit_message='Upload model', commit_description='', oid='9c72d7dcc7609aa7bf7c0542aa6ba85eccea8a72', pr_url=None, pr_revision=None, pr_num=None)